<a href="https://colab.research.google.com/github/Yuuu553/-/blob/main/%E8%A4%87%E8%B3%BD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install gspread oauth2client bcrypt snownlp pytz speechrecognition
!pip install gradio

import gradio as gr
import bcrypt
import json
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
from snowballstemmer import stemmer
import speech_recognition as sr

# Google Sheets 設置
SCOPE = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
CREDS_FILE = "metal-node-452506-c4-6df5ac62a6e9.json"
SPREADSHEET_NAME = "EmotionDiary"

creds = ServiceAccountCredentials.from_json_keyfile_name(CREDS_FILE, SCOPE)
client = gspread.authorize(creds)
sheet = client.open_by_key('1AT-e76Sqi1RsjOftc--kjN5jZXi3tDT6cghwdlUbM9U').sheet1


# 用戶認證
USERS_FILE = "users.json"

def load_users():
    if os.path.exists(USERS_FILE):
        with open(USERS_FILE, "r") as f:
            return json.load(f)
    return {}

def save_users(users):
    with open(USERS_FILE, "w") as f:
        json.dump(users, f)

def register(username, password):
    users = load_users()
    if username in users:
        return "用戶已存在！"
    hashed_pw = bcrypt.hashpw(password.encode(), bcrypt.gensalt()).decode()
    users[username] = hashed_pw
    save_users(users)
    return "註冊成功！"

def login(username, password):
    users = load_users()
    if username in users and bcrypt.checkpw(password.encode(), users[username].encode()):
        return "登入成功！"
    return "帳號或密碼錯誤！"

# 儲存情緒記錄
def save_entry(username, mood, note):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    sheet.append_row([username, timestamp, mood, note])
    return "紀錄已儲存！"

# 查詢情緒記錄
def view_entries(username):
    records = sheet.get_all_records()
    user_records = [r for r in records if r['username'] == username]
    return "\n".join([f"{r['timestamp']} - {r['mood']}: {r['note']}" for r in user_records])

# 語音轉文字
def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        try:
            audio = recognizer.listen(source)
            text = recognizer.recognize_google(audio, language='zh-TW')
            return text
        except sr.UnknownValueError:
            return "無法識別語音"
        except sr.RequestError:
            return "請求錯誤"

# Gradio UI
with gr.Blocks() as app:
    gr.Markdown("# 情緒日記")
    username = gr.Textbox(label="帳號")
    password = gr.Textbox(label="密碼", type="password")
    login_btn = gr.Button("登入")
    register_btn = gr.Button("註冊")
    login_output = gr.Textbox(label="狀態")

    login_btn.click(login, inputs=[username, password], outputs=login_output)
    register_btn.click(register, inputs=[username, password], outputs=login_output)

    with gr.Row():
        mood = gr.Dropdown(["開心", "難過", "生氣", "焦慮", "平靜"], label="你的心情？")
        note = gr.Textbox(label="想說的話")
        save_btn = gr.Button("儲存")
    save_output = gr.Textbox(label="狀態")
    save_btn.click(save_entry, inputs=[username, mood, note], outputs=save_output)

    view_btn = gr.Button("查看紀錄")
    records_output = gr.Textbox(label="你的紀錄")
    view_btn.click(view_entries, inputs=[username], outputs=records_output)

    stt_btn = gr.Button("語音輸入")
    stt_output = gr.Textbox(label="轉換文字")
    stt_btn.click(speech_to_text, outputs=stt_output)

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a3e6cca949de00bf00.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
